In [4]:
import pandas as pd
import tqdm
from datasets import load_dataset
from typing import List, Dict

from langchain import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

## Vectorstore 구축

In [57]:
dataset = load_dataset("WiktorS/polish-news")
df = pd.DataFrame(dataset['train'][:50])
df['text'] = df.apply(lambda x: f"{x['title'].strip()} {x['headline'].strip()} {x['content'].strip()}", axis=1)
df = df[['link', 'text']]
df = df.to_dict(orient='records')

In [59]:
def _embedd_documents(docs: List[str], metadata: List[Dict], vectorstore_filepath: str):
    # Creating vectorstore
    embeddings=HuggingFaceBgeEmbeddings()
    vectorstore = FAISS.from_texts(docs, embeddings, metadatas=metadata)
    with open(vectorstore_filepath, 'wb') as f:
        pickle.dump(vectorstore, f)

In [60]:
def process_data(data: List[Dict[str, str]], vectorstore_filepath: str):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50,
        length_function=len
    )

    # Split documents into chunks
    processed_docs, metadata = [], []
    for doc in tqdm(data):
        chunks = text_splitter.split_text(doc['text'])
        processed_docs.extend(chunks)
        metadata.extend([{"source": doc['link']}] * len(chunks))

    _embedd_documents(processed_docs, metadata, vectorstore_filepath)

In [61]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len
)

In [62]:
processed_docs, metadata = [], []
for doc in df:
    chunks = text_splitter.split_text(doc['text'])
    processed_docs.extend(chunks)
    metadata.extend([{"source": doc['link']}] * len(chunks))

In [75]:
metadata

[{'source': 'https://www.tvp.info/68305505/w-holandii-krowy-sikaja-zlotem-tylko-w-specjalnych-toaletach'},
 {'source': 'https://www.tvp.info/68305505/w-holandii-krowy-sikaja-zlotem-tylko-w-specjalnych-toaletach'},
 {'source': 'https://www.tvp.info/68305505/w-holandii-krowy-sikaja-zlotem-tylko-w-specjalnych-toaletach'},
 {'source': 'https://www.tvp.info/68305505/w-holandii-krowy-sikaja-zlotem-tylko-w-specjalnych-toaletach'},
 {'source': 'https://www.tvp.info/68305505/w-holandii-krowy-sikaja-zlotem-tylko-w-specjalnych-toaletach'},
 {'source': 'https://www.tvp.info/68303268/plan-chin-piecioprocentowy-wzrost-gospodarczy'},
 {'source': 'https://www.tvp.info/68303268/plan-chin-piecioprocentowy-wzrost-gospodarczy'},
 {'source': 'https://www.tvp.info/68303268/plan-chin-piecioprocentowy-wzrost-gospodarczy'},
 {'source': 'https://www.tvp.info/68303268/plan-chin-piecioprocentowy-wzrost-gospodarczy'},
 {'source': 'https://www.tvp.info/68303268/plan-chin-piecioprocentowy-wzrost-gospodarczy'},
 {'so

In [72]:
embeddings=HuggingFaceBgeEmbeddings()
vectorstore = FAISS.from_texts(processed_docs, embeddings, metadatas=metadata)

## 우리꺼로 Dataset 구축

In [7]:
old_dataset = pd.read_csv('./data/data.csv')
old_dataset

,link,text
0,https://www.tvp.info/68305505/w-holandii-krowy...,W Holandii krowy „sikają złotem”. Tylko w spec...
1,https://www.tvp.info/68303268/plan-chin-piecio...,Chiny obniżyły prognozy dotyczące wzrostu gosp...
2,https://www.tvp.info/68292477/ogromny-wzrost-w...,Ogromny wzrost wpływów z VAT. Ministerstwo fin...
3,https://www.tvp.info/68289222/europejski-dzien...,Niemcy. Tylko w jednej branży kobiety zarabiaj...
4,https://www.tvp.info/68289414/moskwa-nie-planu...,Moskwa nie planuje naprawy Nord Stream 1 i 2. ...
5,https://www.tvp.info/68280509/wzrosty-kursow-a...,Na Wall Street wreszcie wzrosty głównych indek...
6,https://www.tvp.info/68272392/tym-razem-tusk-u...,Tym razem Tusk uderza w program Mieszkanie Plu...
7,https://www.tvp.info/68270122/zakaz-sprzedazy-...,Szlaban na sprzedaż cywilnych dronów w sieci. ...
8,https://www.tvp.info/68271034/bezrobocie-w-kra...,Bezrobocie w UE. Polska z jednym z najniższych...
9,https://www.tvp.info/68268733/prezydent-andrze...,Prezydent Andrzej Duda czeka na propozycje ins...


In [5]:
dataset = pd.read_csv('./data/panre_text_cleaned.csv')
dataset

,판례정보일련번호,사건명,사건번호,선고일자,선고,법원명,법원종류코드,사건종류명,사건종류코드,판결유형,판시사항,판결요지,참조조문,참조판례,판례내용
0,239461,손해배상(기),2019다208724,20240215.0,선고,대법원,400201.0,민사,400101.0,판결,[1] 물건에 대한 점유의 의미 및 판단 기준 / 민법 제758조 제1항에서 정...,[1] 물건에 대한 점유란 사회관념상 어떤 사람의 사실적 지배에 있다고 보이는 ...,"[1] 민법 제192조, 제195조, 제758조 제1항 / [2] 민법 제192조...","[1] 대법원 2001. 1. 16. 선고 98다20110 판결(공2001상, 4...","【원고, 피상고인 겸 상고인】 주식회사 서영엔지니어링\n【원고(선정당사자), 피상고..."
1,239463,대상청구의소,2019다238640,20240215.0,선고,대법원,400201.0,민사,400101.0,판결,수익자가 사해행위취소 소송의 확정판결에 따른 원상회복으로 대체물 인도의무를 이행...,"민법 제395조에 따르면, 채무자가 채무의 이행을 지체한 경우에 채권자가 상당한 기...","민법 제395조, 제406조 제1항, 제407조\n","대법원 1975. 5. 13. 선고 75다308 판결(공1975, 8461), 대법...","【원고, 피상고인】 신용보증기금 (소송대리인 법무법인 법여울 담당변호사 박영만 외 ..."
2,239451,부당이득금,2023다295442,20240215.0,선고,대법원,400201.0,민사,400101.0,판결,[1] 토지 소유자가 소유 토지를 일반 공중 등의 통행로로 무상 제공하거나 그에...,[1] 어느 사유지가 종전부터 자연발생적으로 또는 도로예정지로 편입되어 사실상 ...,"[1] 헌법 제23조 제1항, 제3항, 민법 제2조, 제211조, 제741조, 제...","[1] 대법원 2013. 8. 22. 선고 2012다54133 판결(공2013하,...","【원고, 상고인】 주식회사 코리아누수설비 (소송대리인 법무법인(유한) 국제 담당변호..."
3,239457,특별대리인선임,2023마7226,20240215.0,자,대법원,400201.0,민사,400101.0,결정,"민사소송법 제64조, 제62조 제1항에 따른 특별대리인 선임신청의 수소법원(=본안사...","법인이 당사자인 경우 이해관계인 등은, 법인의 대표자가 없거나 대표자에게 소송에...","민사소송법 제62조 제1항, 제64조\n",NaN,【재항고인】 재항고인\n【원심결정】 광주고법 2023. 9. 5. 자 (전주)202...
4,239455,구상금,2023다272883,20240215.0,선고,대법원,400201.0,민사,400101.0,판결,[1] 무보험자동차에 의한 상해 담보특약의 법적 성질(=손해보험형 상해보험) 및...,[1] 피보험자가 무보험자동차에 의한 교통사고로 인하여 상해를 입었을 때 그 손...,"[1] 상법 제665조, 제672조 제1항, 제737조, 민법 제105조 / [2...","[1] 대법원 2002. 5. 17. 선고 2000다30127 판결(공2002하,...","【원고, 피상고인】 현대해상화재보험 주식회사 (소송대리인 법무법인 도원 담당변호사 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87062,85831,토지건물이전등기,4280민상383,19480527.0,선고,대법원,400201.0,민사,400101.0,판결,가. 매매계약합의해제와 합의해제후 매매대금수수 양사실 변존의 주장은 모순아닌가\n나...,가. 매매계약합의해제와 합의해제후 매매대금수수 양사실의 변존을 인정함은 사회통념에 ...,NaN,NaN,"【원고, 상고인】 박종식\n【피고, 피상고인】 윤엽\n【주 문】\n ..."
87063,85836,토지소유권이전등기,4281민상417,19480412.0,선고,대법원,400201.0,민사,400101.0,판결,출계한 양자와 생부사망에 의한 소송수계의 적부\n,타인의 양자로 출계한 자는 그 생부의 상속인이라 할 수 없음으로 생부가 사망한 경...,민사소송법 제208조\n,NaN,"【원고, 상고인】 \n【피고, 피상고인】 \n【원심판결】 \n서울고등법원 1948...."
87064,85835,부동산소유권이전등기,4281민상362,19480407.0,선고,대법원,400201.0,민사,400101.0,선고,가. 소송위임 흠결의 항변과 당해 이원의 인증\n나. 계약해제존재의 가항변에 대한 ...,가. 소송위임장이 사문서인 경우에 당사자 일방이 그 소송위임의 흠결을 주장한 때에는...,"민사소송법 제80조\n, \n 제137조, \n 제395조\n",NaN,"【원고, 피상고인】 안유길\n【피고, 상고인】 최석희\n【원심판결】 \n서울..."
87065,85834,토지건물소유권이전등기,4281민상298,19480402.0,선고,대법원,400201.0,민사,400101.0,판결,잔대금 지불후에 이전등기절차를 이행하기로한 약지와 동시이행과의 관계\n,매매계약에서 잔대금의 지불을 받은 후에 이전등기의무를 행하기로 약정한 경우의 약지...,NaN,NaN,"【원고, 상고인】 정석용\n【피고, 피상고인】 송종순\n【원심판결】 \n청주..."


## QA module

In [96]:
class AnsweringModel:
    def __init__(self):
        vectorstore_path = './data/myvectorstore.pkl'

        with open(vectorstore_path, "rb") as f:
            self.store = pickle.load(f)

        self.model = 
        self.prompt = hub.pull("rlm/rag-prompt")

        self.chain = RetrievalQAWithSourcesChain.from_chain_type(chain_type='stuff',
                                                                 llm=self.model,
                                                                 retriever=self.store.as_retriever(),
                                                                 prompt=self.prompt)

    def answer(self, query: str) -> Tuple[str, str]:
        result = self.chain({"question": query})
        return result['answer'].strip(), result['sources']

In [107]:
prompt = hub.pull('rlm/rag-prompt')
prompt

ChatPromptTemplate(input_variables=['question', 'context'], output_parser=None, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question', 'context'], output_parser=None, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:", template_format='f-string', validate_template=True), additional_kwargs={})])

In [1]:
import pickle
from transformers import AutoModelForCausalLM, AutoTokenizer

class AnsweringModel:
    def __init__(self):
        vectorstore_path = './data/myvectorstore.pkl'
        with open(vectorstore_path, "rb") as f:
            self.store = pickle.load(f)
        
        self.model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")
        self.retriever = self.store.as_retriever(search_type='similarity')
        self.prompt = "{question} What do the documents say about it? {docs}"
        self.tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
    
    @staticmethod
    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    def answer_question(self, question, max_docs=2):
        relevant_texts = self.retriever.get_relevant_documents(question)
        relevant_texts = relevant_texts[:max_docs]
        formatted_docs = self.format_docs(relevant_texts)
        input_prompt = self.prompt.format(question=question, docs=formatted_docs)
        input_ids = self.tokenizer.encode(input_prompt, return_tensors='pt')
        outputs = self.model.generate(input_ids, max_new_tokens=300)
        answer = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        return answer, formatted_docs

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
query = 'Kto jest premierem Niderlandów?'
answer, sources = AnsweringModel().answer_question(query)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [14]:
print(answer)

Kto jest premierem Niderlandów? What do the documents say about it? Marka Ruttego zdecydował w ubiegłym roku, że Niderlandy do 2030 roku zredukują liczbę emisji tlenku azotu o 70 procent. Media ujawniły ekspertyzy, z których wynika, że konieczna będzie redukcja o jedną trzecią zwierząt hodowlanych w holenderskich fermach. Na tym tle doszło do rolniczych protestów na terenie całego kraju. Blokowane były autostrady, a także centra dystrybucyjne supermarketów.W ubiegły piątek protestowali także rolnicy u południowego sąsiada. Przeszło 2,7 tys.

W Holandii krowy „sikają złotem”. Tylko w specjalnych toaletach W Holandii powstaje coraz więcej toalet dla krów – informuje dziennik „Algemeen Dagblad”. Mogą one pomóc w rozwiązaniu problemu z emisją szkodliwego tlenku azotu, ale też – jak zauważa gazeta – mogą stać się źródłem zysków dla hodowców bydła. „Krowa sika nowym płynnym złotem?” – tak zatytułował niedawno swój artykuł popularny dziennik Niderlandów. Wykorzystywanie krowi

In [15]:
print(sources)

Marka Ruttego zdecydował w ubiegłym roku, że Niderlandy do 2030 roku zredukują liczbę emisji tlenku azotu o 70 procent. Media ujawniły ekspertyzy, z których wynika, że konieczna będzie redukcja o jedną trzecią zwierząt hodowlanych w holenderskich fermach. Na tym tle doszło do rolniczych protestów na terenie całego kraju. Blokowane były autostrady, a także centra dystrybucyjne supermarketów.W ubiegły piątek protestowali także rolnicy u południowego sąsiada. Przeszło 2,7 tys.

W Holandii krowy „sikają złotem”. Tylko w specjalnych toaletach W Holandii powstaje coraz więcej toalet dla krów – informuje dziennik „Algemeen Dagblad”. Mogą one pomóc w rozwiązaniu problemu z emisją szkodliwego tlenku azotu, ale też – jak zauważa gazeta – mogą stać się źródłem zysków dla hodowców bydła. „Krowa sika nowym płynnym złotem?” – tak zatytułował niedawno swój artykuł popularny dziennik Niderlandów. Wykorzystywanie krowiego moczu jako zamiennika sztucznego nawozu nie jest w Holandii


# 데이터셋 확인

In [22]:
import json

with open('./data_eval/a_ha.json', 'r') as f:
    data = json.load(f)

data

{'0': {'urls': 'https://www.a-ha.io/questions/4cb726a43f6fea7f8f6b2836fd425c45',
  'title': '상법에서 정한 기한을 넘어선 주주총회의사록 공증 진행시 효력이 있는지요?',
  'question': '안녕하세요,주주총회를 진행한 이후 일정 기간안에 법무사를 통한 공증을 진행해야 하는것으로 알고 있습니다.그러나 현재 각 주주들의 개인인감 등 준비서류가 기간 내 진행이 힘들것으로 사료됩니다.만일 법에서 정한 기한 내 공증이 이루어지지 않으면 법적으로 효력을 상실하게 되나요?아니면, 지나더라도 우선 공증을 진행하고 주주들의 소 제기가 없다면 유야무야 끝나게 되는것인지요?',
  'answer': ['안녕하세요. 이성재 변호사입니다.주주총회 결의사항 중 등기사항이 없다면 공증을 받지 않아도 됩니다. 그러나 등기사항이 있다면 반드시 공증이 필요합니다. 결의사항 중 등기사항이 있음에도 공증을 받지 않아 등기절차를 게을리하거나, 등기를 누락하면 상법 제 635조에 따라 해당 법인 대표는 500만원 이하의 과태료 처분을 받을 수도 있습니다. 이 때 과태료를 내야 될 사람은 법적으로 법인이 아닌 대표자 개인입니다. 따라서 회사의 비용으로 법인등기 과태료를 처리할 수 없습니다.',
   '주주총회 회의록에 대해서 상법에서 정하고 있는 공증 기한을 도과한 경우,해당 대표에 대해서 과태료 처분이 내려질 수 있으나 추후에라도 공증이 이루어진다면 그러한 주주총회 자체가 부적법해지는 것은 아닙니다']},
 '1': {'urls': 'https://www.a-ha.io/questions/4a8035d424b93c9eb4aa0687050075b3',
  'title': '형사사법포털에는 기소유예가 떴는데 아직 어떠한 연락도 안와서요 혹시 왜그런건가요?',
  'question': '점유이탈횡령죄로 조사받고 얼마전 2일 기소유예 처분을 받았습니다. 근데 연락 받아서 처분 결과를 안것이 아니고 형사사법포털 앱을 통해 제가 안

In [24]:
import pandas as pd

rows = []
for key, value in data.items():
    if isinstance(value, dict) and {'urls', 'title', 'question', 'answer'} <= value.keys():
        query = value['title'] + " " + value['question']
        gold_answer = value['answer']
        rows.append({'query': query, 'gold_answer': gold_answer})

df = pd.DataFrame(rows)
df['gold_answer'] = df['gold_answer'].str[2:-2]
df.to_csv('./data_eval/a_ha_evaluation.csv', index=False)

In [26]:
df

,query,gold_answer
0,상법에서 정한 기한을 넘어선 주주총회의사록 공증 진행시 효력이 있는지요? 안녕하세요...,안녕하세요. 이성재 변호사입니다.주주총회 결의사항 중 등기사항이 없다면 공증을 받지...
1,형사사법포털에는 기소유예가 떴는데 아직 어떠한 연락도 안와서요 혹시 왜그런건가요? ...,안녕하세요? 아하(Aha) 법률 상담 지식답변자 김성훈 변호사입니다. 질문하신 내용...
2,"자본금 10억 미만의 법인회사에서 이사가 1명일 때 발생하는 문제점 안녕하세요,자본...","10억 미만의 회사이고 이사가 과반주주권자라면,주주총회 등을 적법하게 통지하고 개최..."
3,모든 계약은 법적으로 15일이내 해지가 가능한지요~? 한 회사(판매 사업)에 지사 ...,모든 계약에 대해서는 15일 만에 해지가 가능하다고 하는 규정은 따로 없습니다해당 ...
4,항소심 재판은 어떻게 진행되는건가요? 아는 지인이 항소후에 항소심 재판이 열리는데 ...,안녕하세요? 아하(Aha) 법률 상담 지식답변자 김성훈 변호사입니다. 질문하신 내용...
...,...,...
111332,전세보증금 선순위에 대해서 질문하고싶습니다 현재 1억8천짜리 전세에 살고 있는데요 ...,안녕하십니까월드클래스 법률사무소의 이승환 대표변호사입니다전입신고와 확정일자를 받았고...
111333,공증이란 것이 법적으로 효력이 있는 것인가요? 친한 친구가 공증을 받고 1300만원...,안녕하십니까월드클래스 법률사무소의 이승환 변호사입니다.공증과 관련하여 사서증서 인증...
111334,경매로 인한 채권회수에 관한 내용입니다. 답변 부탁드립니다. 전세로 살던집이 경매로...,법무법인 에스엔의 홍경열 변호사입니다임차인은 전입신고와 확정일자를 갖춘경우 그 일자...
111335,등기의 공신력 문의 드립니다. 우리나라는 부동산등기에 대한 공신력을 인정 받지 못하...,법무법인 에스엔의 홍경열 변호사입니다질문자께서 말씀하신바와 같이 부동산등기에는 공신...
